In [1]:
import os, io, json, transformers, pinecone, fitz, pypdf, faiss, sqlite3, langchain_community, langchain, openai, math, time 
import requests
from transformers import pipeline
import pandas as pd
import numpy as np
from io import StringIO
from dotenv import load_dotenv
from operator import itemgetter

from langchain import document_loaders, embeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
#from llama_index.core.node_parser import SentenceSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pinecone import Pinecone, ServerlessSpec, Pinecone         # vector store

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.schema import TextNode
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

# sentence transformers
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/Users/talktalki/miniconda3/envs/genAIlab_hw2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

access_endpoint_api_key =os.getenv('access_endpoint_api_key')
openai_key = os.getenv('OPENAI_API_KEY')
#pinecone_api_key =os.getenv('PINECONE_API_KEY')
environment =os.getenv('PINECONE_ENV')
HF_TOKEN = os.getenv('HF_TOKEN')

# configure Pinecone client
pinecone_api_key = '1b7fc13c-0ea1-4edc-a174-244bce3e0b47'
pc = Pinecone(api_key=pinecone_api_key)

## **Part 1: Product Selection and Customer Review Data Collection**

In [3]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
import time

def get_page(url, headers):
    """
    Get webpage results into a Beautiful Soup object.
    """
    try:
        req = Request(url, headers=headers)
        page = urlopen(req)
        soup = BeautifulSoup(page, "html.parser")
        return soup
    except HTTPError as e:
        print(f"HTTP error: {e.code} {e.reason}")
    except URLError as e:
        print(f"URL error: {e.reason}")
    except Exception as e:
        print(f"General error: {e}")
    return None

def scrape_product_details(base_url, product_id, headers):
    """
    Scrape product description and up to 200 most recent reviews.
    """
    url = f"{base_url}/dp/{product_id}/"
    print("page url:", url)
    soup = get_page(url, headers)
    if not soup:
        return None, []

    # Find product description
    try:
        #description = soup.find_all('div', id='productDescription', class_="a-section a-spacing-small").get_text(strip=True)
        description = soup.find('div', id='feature-bullets').get_text(strip=True)
    except AttributeError:
        description = "No description available"

    reviews = []
    # Adjust the URL for 'Most Recent' reviews and iterate through pages
    review_url = f"{base_url}/product-reviews/{product_id}/"
    print("review url:", review_url)
    for i in range(1, 21):  # Assume there are 10 reviews per page, fetch 20 pages
        url = f"{review_url}?pageNumber={i}&sortBy=helpful"
        soup = get_page(url, headers)
        if not soup:
            continue
        review_divs = soup.find_all('div', {'data-hook': 'review'})
        for div in review_divs:
            try:
                #title = div.find('a', {'data-hook': 'review-title'}).get_text(strip=True)
                body = div.find('span', {'data-hook': 'review-body'}).get_text(strip=True)
                reviews.append(body)
                if len(reviews) >= 200:
                    break
            except AttributeError:
                continue
        if len(reviews) >= 200:
            break
        time.sleep(1)  # Sleep to prevent too frequent requests

    return description, reviews

# Usage example
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
urls = {"https://www.amazon.com/Samsonite-Omni-Hardside-Spinner-Teal": "B013WFNZRE",
        "https://www.amazon.com/Hurtle-3-Wheeled-Scooter-Kids-Graffiti": "B08BDGS58Q",
        "https://www.amazon.com/DeLonghi-ECAM29084SB-Magnifica-LatteCrema-Espresso": "B0B38KRTV6"
        }

descriptions = []
reviews = []
for base_url, product_id in urls.items():
    description, review = scrape_product_details(base_url, product_id, headers)
    descriptions.append(description)
    reviews.append(review)

products = ["Product A", "Product B", "Product C"]

product_data = {}
for i in range(len(products)):
    product_data[products[i]] = {
        "description": descriptions[i],
        "reviews": reviews[i]
    }
print(product_data)

page url: https://www.amazon.com/Samsonite-Omni-Hardside-Spinner-Teal/dp/B013WFNZRE/
review url: https://www.amazon.com/Samsonite-Omni-Hardside-Spinner-Teal/product-reviews/B013WFNZRE/
page url: https://www.amazon.com/Hurtle-3-Wheeled-Scooter-Kids-Graffiti/dp/B08BDGS58Q/
review url: https://www.amazon.com/Hurtle-3-Wheeled-Scooter-Kids-Graffiti/product-reviews/B08BDGS58Q/
page url: https://www.amazon.com/DeLonghi-ECAM29084SB-Magnifica-LatteCrema-Espresso/dp/B0B38KRTV6/
review url: https://www.amazon.com/DeLonghi-ECAM29084SB-Magnifica-LatteCrema-Espresso/product-reviews/B0B38KRTV6/
{'Product A': {'description': '24" SPINNER LUGGAGE maximizes your packing power and is the ideal checked bag for longer tripsPACKING Dimensions: 24” x 17.5” x 11.5”, Overall Dimensions: 26.5” x 17.75” x 11.75”, Weight: 8.34 lbs.10 YEAR LIMITED WARRANTY: Samsonite products are rigorously tested to ensure our products meet stringent standards. This bag comes with a 10-year warranty against defects in materials a

In [4]:
product_data

{'Product A': {'description': '24" SPINNER LUGGAGE maximizes your packing power and is the ideal checked bag for longer tripsPACKING Dimensions: 24” x 17.5” x 11.5”, Overall Dimensions: 26.5” x 17.75” x 11.75”, Weight: 8.34 lbs.10 YEAR LIMITED WARRANTY: Samsonite products are rigorously tested to ensure our products meet stringent standards. This bag comes with a 10-year warranty against defects in materials and workmanship.MICRO-DIAMOND POLYCARBONATE texture is extremely scratch-resistant, keeping cases beautiful trip after tripSIDE-MOUNTED TSA LOCKS act to deter theft, ensuring that only you or a TSA agent have easy access to your belongings when travelingFOUR, MULTI-DIRECTIONAL OVERSIZED SPINNER WHEELS for effortless mobility, re-engineered lightweight',
  'reviews': ['Perfect weight and very maneuverable.  Zippers work easily and has a convenient TSA approved lock.  Looks great and will be perfect size for longer trips.  Happy with the purchase / great value for the money.',
   'Wh

In [48]:
nodes = []
i = "Product B"

# Create a TextNode for the description
desc_node = TextNode(text=product_data[i]['description'])
nodes.append(desc_node)

# Create a TextNode for each review
for review in product_data[i]['reviews']:
    review_node = TextNode(text=review)
    nodes.append(review_node)

In [49]:
gpt_3_5_turbo = "gpt-3.5-turbo"
llm = OpenAI(model=gpt_3_5_turbo)

extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
]

pipeline = IngestionPipeline(
    transformations=extractors,
)

nodes = await pipeline.arun(nodes=nodes, in_place=False)

TypeError: OpenAI.__init__() got an unexpected keyword argument 'model'

In [7]:
embed_model_3_small="text-embedding-3-small"
embed_model = OpenAIEmbedding(model=embed_model_3_small, openai_api_key=openai)

# create embeddings for the text dataset
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [8]:
use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

if use_serverless:
    spec = pinecone.ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = pinecone.PodSpec(environment=environment)

# Name our Pinecone Index
index_name = "hw03"

# If a Pinecone index of the same name already exists, delete it:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [9]:
# Name our Pinecone Index
index_name = "hw03"

dimensions = 1536 #768              # the dimensions of the index need to align with the LLM we are using for the RAG system. For example, if using openAI then dimenion = 1536. If using Llama2, then dimension = 384.

pc.create_index(
    name=index_name, 
    dimension=dimensions, 
    metric="cosine",          # we can use different distance metrics to measure the similarity between vector embeddings and user queries. this is where we define what similarity metric we are going to use for the vector store.
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

for index in pc.list_indexes():
    print(index['name'])

pc.describe_index("hw03")

pc_index = pc.Index(index_name)  # create an index to use in the vector store

vector_store = PineconeVectorStore(pinecone_index=pc_index)    # this function creates a vector store where we will add and store embeddings

hw03


In [10]:
# nothing so far - empty index
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [11]:
vector_store.add(nodes)

Upserted vectors: 100%|██████████| 101/101 [00:04<00:00, 22.66it/s]


['1c449ca9-e5e5-49bb-8b8e-d50d7f5d4dfe',
 '061fd5e6-9abe-4514-b9d8-cc0d94d27266',
 '1fb247e5-0285-49b6-b1d4-088cfa5ff425',
 '3de3c5af-71d1-4472-a049-119104c41710',
 '5953d0e5-04be-44b5-afd5-d2ba1f7210a6',
 'c0e08b10-39af-4745-bad6-79ac10c82a3f',
 '97d7ac1e-9398-4511-82a5-8d564429c773',
 '23bf9240-b2c4-40db-a6b5-bc401eb397a5',
 '23e73ebd-c1c8-42bc-831c-dc50adb7ba3d',
 '943edc55-3593-44cd-877c-fb8a30fd7319',
 '5eec977d-4714-46fa-a634-3c890255bd62',
 '01e74c20-1094-4f5d-95f1-151d8fb37cfe',
 'd6b0d14b-a8ca-4035-9b29-4b321b5d7caa',
 'de29791c-514b-4ebb-a843-2343e5a95047',
 '65a1431a-5582-48c9-a3b7-0beeb6854752',
 '68483375-5f5a-47ba-b846-91fc3980102a',
 '6c654593-31c5-4b88-a9d6-c0243ab5e396',
 '116e6e1b-4cc0-4d7e-add8-fa19517c827d',
 '60c67f3f-dfbd-4686-a854-7cd8daced33c',
 'e8e6e210-d3f4-46b7-b024-68645a651879',
 'd038f1d3-1aaf-4c93-b8d9-fe828235e8e0',
 'c912afb3-57ef-4c36-818c-bc2a3f05e399',
 '5cd50247-10a3-479a-97d3-60202676a8ba',
 '3994ca10-01df-4129-bb2c-d4e4f8ad0083',
 '23bf64ba-cc89-

In [12]:
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [13]:
print(nodes[0].metadata)
print(nodes[0])

{'document_title': '"Samsonite Spinner Luggage: Effortless, Durable, and Stylish Travel Options with TSA Locks and Micro-Diamond Polycarbonate Texture"', 'questions_this_excerpt_can_answer': '1. What are the dimensions and weight of the 24" Samsonite Spinner Luggage?\n2. What type of warranty does the Samsonite Spinner Luggage come with and for how long?\n3. What features does the Samsonite Spinner Luggage have to ensure security and ease of mobility during travel?'}
Node ID: 1c449ca9-e5e5-49bb-8b8e-d50d7f5d4dfe
Text: 24" SPINNER LUGGAGE maximizes your packing power and is the
ideal checked bag for longer tripsPACKING Dimensions: 24” x 17.5” x
11.5”, Overall Dimensions: 26.5” x 17.75” x 11.75”, Weight: 8.34
lbs.10 YEAR LIMITED WARRANTY: Samsonite products are rigorously tested
to ensure our products meet stringent standards. This bag comes with a
10-year warr...


In [23]:
def get_sys_message(q, k):
    # get embedding
    result = client.embeddings.create(
        input=[q],
        model=embed_model_3_small
    )

    # retrieve from Pinecone vector store
    xq = result.data[0].embedding

    # search
    res2 = pc_index.query(vector=xq, top_k=k, include_metadata=True)
    
    context = ""
    for match in res2['matches']:
        dict_obj = json.loads(match['metadata']['_node_content'])
        context += dict_obj['text'] + " "
    context = context.replace("\n", "")

    return context

In [29]:
queries = ["Describe in detail what this product is, based on the product description.",
           "What are some features of this product, especially focusing on visual features?",
           "What is the main purpose of this product?",
           "What is most loved about this product?",
           "What is most hated about this product?"
]   

In [30]:
# try a query
get_sys_message(q=queries[0], k=20)

'Very good product Seller was fantastic, great luggage Great product and love the look. Have not had this for long to give a proper review, but so far happy with purchased. Will update if needed. Just like I expected 👌 however the wheels could work better Nothing I haven\'t used this yet but looking it over I\'m very happy with my purchase. Needs a zipper separation for one side and an accessory pocket. I got this suitcase for traveling, love it!  Samsonite has the very best quality and I knew I could count on getting a top notch product. I received it in a timely manner as well. It did not include the key to create the password Perfect My son , and his girlfriend told me about this suit case. They love it when they go on their trips Traveling Travel Haven’t used yet, but I’m happy so far with purchase.  Like darker color without being black.  Good size with expandable sides. Now that I\'ve had the chance to travel with this, I can finally give my full review. I purchased the carry-on 

In [31]:
def rag_openAI_gpt(model, q, k, prompt):
    client = OpenAI()
    f=get_sys_message(q, k)
    
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "Instruction: use the information in" + f + "to answer the user's question."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": f},
        {"role": "user", "content": "What is the answer?"}
    ]
    )
    return response.choices[0].message.content

In [44]:
import logging

In [46]:
# LOGGER
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='log/test.log')

In [37]:
# Run
analyses = {}
for i in range(len(queries)):
    response = rag_openAI_gpt(model=gpt_3_5_turbo, q=queries[i], k=20, prompt="Product Description")
    print(f"<Query: {queries[i]}>")
    print(response)
    analyses[i+1] = response
    print()
analyses

<Query: Describe in detail what this product is, based on the product description.>
Based on the reviews provided, it seems that the product being described is a Samsonite hardshell suitcase in a "Caribbean Blue" color. The suitcase is praised for its noticeable color, sturdy design, TSA-approved lock system, expandable storage capacity, and durability. Some users mentioned that the wheels could work better at times, and a zipper separation for one side and an accessory pocket would be useful additions. Overall, it is highlighted as a great carry-on option that fits a lot despite its size, making it suitable for traveling.

<Query: What are some features of this product, especially focusing on visual features?>
Based on the feedback provided by users, the product in question appears to be a Samsonite hard-sided expandable luggage with spinner wheels. Users are generally satisfied with their purchase, highlighting features such as the expandable sides, good size, durability, roominess, 

{1: 'Based on the reviews provided, it seems that the product being described is a Samsonite hardshell suitcase in a "Caribbean Blue" color. The suitcase is praised for its noticeable color, sturdy design, TSA-approved lock system, expandable storage capacity, and durability. Some users mentioned that the wheels could work better at times, and a zipper separation for one side and an accessory pocket would be useful additions. Overall, it is highlighted as a great carry-on option that fits a lot despite its size, making it suitable for traveling.',
 2: 'Based on the feedback provided by users, the product in question appears to be a Samsonite hard-sided expandable luggage with spinner wheels. Users are generally satisfied with their purchase, highlighting features such as the expandable sides, good size, durability, roominess, distinctive colors, and maneuverability. Some users also mention that the wheels could work better, particularly in terms of handling heavy loads or walking too f

In [47]:
logging.info(analyses)

In [40]:
analyses = {
    "Product Description": analyses[1],
    "Product Features": analyses[2],
    "Product Usage": analyses[3],
    "Product Sentiment Good": analyses[4],
    "Product Sentiment Bad": analyses[5]
}

## **Part 3: Image Generation**

In [41]:
prompt = f"""   
        Generate 3-5 images for product. 
        Description: {analyses["Product Description"]}. 
        Features: {analyses["Product Features"]}.
        Usage: {analyses["Product Usage"]}. 
        Good Sentiment: {analyses["Product Sentiment Good"]}.
        Bad Sentiment: {analyses["Product Sentiment Bad"]}.
        """

In [43]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True